In [2]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

#Progres bar
from tqdm import tqdm

In [3]:
#Load output.csv into df
with open('output.csv', encoding='utf8') as input_file:
    df = pd.read_csv(input_file)

In [4]:
#Generate list of all users
users = df['sender'].unique()

#Create empty graph of users
graph = np.zeros((len(users), len(users)))

#Add an additional edge from user a to user b if user a replies to user b
pbar = tqdm(total=len(df))
for k, row in df.iterrows():
    pbar.update(1)
    if row['reply_to_msg_id'] != -1:
        a = np.where(users == row['sender'])
        repliedMessageID = row['reply_to_msg_id']
        #Check if replied message exists in the df
        if repliedMessageID in df['msg_id'].values:
            #Get the row of the dataframe corresponding to repliedMessageID
            repliedMessageRow = df[df['msg_id'] == repliedMessageID]
            #Get the sender of repliedMessageRow
            repliedMessageSender = repliedMessageRow['sender'].values[0]
            #Get the index of repliedMessageSender in users
            b = np.where(users == repliedMessageSender)
            #Add an edge from a -> b
            graph[a, b] += 1
pbar.close()


print(graph)

In [32]:
leaderboard = []
for x, row in enumerate(graph):
    for y, val in enumerate(row):
        leaderboard.append([val, users[x], users[y]])
#Sort leaderboard by val
leaderboard = sorted(leaderboard, key=lambda x: x[0], reverse=True)
#Print leaderboard with newline after every element, adjusted so columns line up with each other
commonPairings = pd.DataFrame(leaderboard)
commonPairings.columns = ['Messages', 'Pair', 'Pair1']
commonPairings['Pair'] = commonPairings['Pair'] + ' & ' + commonPairings['Pair1']
del commonPairings['Pair1']
reduced = commonPairings.drop(commonPairings.index[10:])
print(reduced)


#print("#", "Original", "Reply", sep="\t")
#for x in leaderboard:
 #   print(x[0], x[1], x[2], sep='\t')

reduced.to_csv("CommonPairings.csv", index=False)

In [6]:
#Create a website with an interactive graph